In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 300)

In [2]:
import json

with open("../data/fix_ipc_class_dict.json") as json_file:
    ipc_class_dict = json.load(json_file)

In [3]:
#df = pd.read_csv("../data/subest1_filtrato.csv")
df = pd.read_csv("../data/data_subset1.csv")

In [4]:
df.title = df.title.str.lower()
df.abstract = df.abstract.str.lower()
df.claims = df.claims.str.lower()
df.ipc_classes = df.ipc_classes.apply(lambda x: x.replace(",", ", ") if type(x)==str else "")

In [5]:
df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US2009216698_A1.txt,2008-02-22,2009-08-27,2008-02-22,"G06F15/18, G06N5/02","XEROX CORPORATIONANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME","ANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME",40999265,temporal events analysis employing tree induction,an events analysis method comprises: optimizing respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k' in branching processes; inferring a most probable branching process for a set of input data co...,1. an events analyzer comprising: a probabilistic branching process learning engine configured to optimize respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k′ in branching processes; and a proba...
1,US2020018760_A1.txt,2019-09-23,2020-01-16,2007-03-27,G01N33/574,IMMUNOVIA,"BORREBAECK, CARL, ARNE, KRISTERWINGREN, LARS, BERTIL, CHRISTER",39493874,protein signature/markers for the detection of adenocarcinoma,the present invention provides a method for determining the presence of pancreatic adenocarcinoma in an individual and/or for determining the survival time of an individual afflicted with pancreatic adenocarcinoma comprising the steps of: (a) providing a serum or plasma sample to be tested; and ...,1. a method for determining the presence of pancreatic adenocarcinoma in an individual comprising the steps of: a) providing a serum or plasma sample to be tested; b) determining a protein signature of the test sample by measuring the presence and/or amount in the test sample of one or more prot...
2,EP3340117_A1.txt,2017-11-15,2018-06-27,2016-12-20,"G06N3/04, G06N3/063",INTEL CORPORATION,"LIN, TSUNG-HAN",60327233,unsupervised learning using neuromorphic computing,"a spiking neural network (snn) is implemented on a neuromorphic computers and includes a plurality of neurons, a first set of the plurality of synapses defining feed-forward connections from a first subset of the neurons to a second subset of the neurons, a second subset of the plurality of syna...","1. a method comprising: defining a spiking neural network (snn) comprising a plurality of artificial neurons interconnected by a plurality of artificial synapses, wherein the snn is to comprise: a first subset of the plurality of synapses to define feedforward connections from a first subset of ..."
3,WO2019070570_A1.txt,2018-10-01,2019-04-11,2017-10-06,A61B5/103,TELLUS YOU CARE,"HSU KEVINCOKE, TANIA",65993331,non-contact activity sensing network for elderly care,"determining a physical state of a person includes detecting positions of different portions of the person, transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions, correlating movement and position data from the ...","1. a method of determining a physical state of a person, comprising:detecting positions of different portions of the person;transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions;correlating movement and positio..."
4,US2016259635_A1.txt,2015-03-04,2016-09-08,2015-03-04,"G06F15/18, G06F17/27, G06F8/65, G06F8/71, G06F9/44, G06F9/445, G06N99/00",IBM (INTERNATIONAL BUSINESS MACHINES CORPORATION),"EL MAGHRAOUI, KAOUTARJANN, JOEFONPATTNAIK, PRATAP C.PICKOVER CLIFFORD A.",56849635,software patch management incorporating sentiment analysis,"a method and system are provided. the method includes generating, by a machine-based sentiment prediction generator, respective machine-determined sentiment predictions for each of a plurality of software patches using sentiment analysis. the method further includes setting, by a sentiment-based...","1. 1-18. (canceled)19. a computer program product for softwar

In [6]:
len(df)

216986

<h3>Keyword della professoressa:</h3>

<img src="keyword prof.png" style="width:800px;"/>

In [7]:
keywords_other = []
"""["molecule",
                  "particle", 
                  "pharmacological",
                  "technolog",
                  "nano(-| )?tech(nolog)?",
                  "alter(ing)?",
                  "nano",
                  "nano(-| )?bot",
                  "nano(-| )?emulsion",
                  "nano(-| )?particle",
                  "nano(-| )?tube",
                  "nano(-| )?sphere",
                  "quantum",
                  "graphene",
                  "cloud",
                  "software",
                  "product"
                 ]"""

"""
Possiamo cercare i vincoli espressi dalla professoressa sia in "stile pandas" 
    (cercare documenti che contengono sia "body" che "care": df[(df.body >= 1) & (df.care >= 1)])
    
    che tramite l'uso delle espressioni regolari per implmentare l'operatore che l'Ing Ghironi 
    ha chiamto NEAR. Ad esempio per cercare "skin NEAR care" possiamo aggiungere alla lista sotto
    "lip(\w|\W){1,20}?stick". Agli estremi ci sono le 2 parole in cui siamo interessati, 
    "(\w|\W){1,20}?" serve invece a dire "permetti la presenza tra 
    0 e 20 caratteri o numeri tra le 2 parole". Se volete fare dei test: https://pythex.org/
"""

keywords_cosmetics = ["cosmetic",
                      "beauty", 
                      "body", 
                      "skin", 
                      "nail", 
                      "fingernail", 
                      "cleans(e|ing)", # 
                      "manicur(e|ing)", #manicure and manicuring
                      "pedicur(e|ing)",
                      "varnish", 
                      "polish",
                      "nail(s| |-)*polish",
                      "lip(s)?",
                      "lip( |-)?stic( |k)+",
                      "blush",
                      "mascara",
                      "make(s| |-)*up",#makes up, make-up
                      "cream",
                      "soap",
                      "toothpaste",
                      "sunscreen",
                      "lotion",
                      "deodorant",
                      "care",
                      "lip(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "nail(s| |-)*(care|polish)",
                      "baby(s| |-)*(care|polish)",
                      "anti(-)?ag(e|ing)",
                      "colour|color",
                      "finger"
                     ]

In [8]:
df_original = df.copy()

### Controllo IPC (DA AFFINARE)
##### IPC A45D+

HAIRDRESSING OR SHAVING EQUIPMENT ; EQUIPMENT FOR COSMETICS OR COSMETIC TREATMENTS, e.g. FOR MANICURING OR PEDICURING

In [9]:
df_selected = pd.DataFrame()

In [10]:
df_candidate = df[(df.ipc_classes.str.count("(^| )A45D")>0)
                 & (df.ipc_classes.str.count("(^| )G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("(^| )A61B5")==0) #	Measuring for diagnostic purposes
                 ]
df_candidate[["title", "ipc_classes","claims"]] 

,title,ipc_classes,claims
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04","1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
31462,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04","1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
38540,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a force ca..."
53114,portable apparatus for supporting a hand held hair dryer,"A45D20/12, A45D20/14, F16M11/00, F16M13/02","1. a portable hair dryer support apparatus, comprising: a hair dryer holding structure for holding a hair dryer; at least one vertical support element pivotally and directly connected at a bottom end to the hair dryer holding structure via a hinge having an axis of rotation perpendicular to the ..."
53136,improvements in and relating to hair irons,"A45D1/06, A45D1/28",1. an electric hair iron comprising at least one heating element and control means comprising a temperature sensor and a control circuit and wherein the control means is arranged such that power is only supplied to the heating element or elements when the temperature sensed by the sensor is at o...
61966,complete compact,"A45D33/00, A45D33/26, A45D44/00, G01N1/22, G01N1/24","i claim:1. a method for determining aroma comprising: (a) providing at least one means of aroma sensing capable of approximately discerning the presence or absence of at least one characteristic of the aroma, (b) sampling aroma with said aroma sensing means, (c) discerning said aroma characteris..."
63517,disposable single use applicator assembly with a chemical composition,"A45D37/00, A45D40/00, A45D40/26, A61K8/04, A61Q19/00","i claim:1. an assembly for storing and transporting, said assembly comprising: a wrapper having a top sheet and a bottom sheet, said top sheet being sealed to said bottom sheet so as to form a border along a perimeter of said top sheet and said bottom sheet; and an insert having a top surface an..."
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
81748,system for targeted application of topical agents to an isolated body part,"A45D40/30, A45D44/00, A61K8/02, A61K8/368, A61P17/10, A61Q19/00, B32B38/10, B32B7/06, B33Y10/00, B33Y80/00, B44C3/02",1. a method for targeted application of topical agents to an isolated

In [11]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

26

##### IPC C11B9/00 (riportato nelle slide come C11B/009)

In [12]:
df_candidate = df[(df.ipc_classes.str.count("(^| )C11B9/00")>0)
                 & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 ]
df_candidate[["title", "ipc_classes","claims"]] 

,title,ipc_classes,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50","1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
135822,method of resolving allergens in perfume ingredients,"B01J20/26, B01J20/281, C11B9/00, G01N30/46, G01N30/86, G16C20/20","1. a method of resolving defined known or suspected allergens in a complex mixture of perfume ingredients using two-dimensional gas chromatography (2d-gc), wherein the combination of first and second stationary phases is selected on the basis of a clustering analysis of defined known or suspecte..."
174990,method of making perfumed goods,"A61L9/01, A61Q19/00, C11B9/00, C11D3/22, C11D3/50",what is claimed is:1. a method comprising:a.) using a malodour reduction value determined by mathematical modelling toselect one or more perfume raw materials;b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product. 2. the me...
210149,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


In [13]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

30

## Analisi Per keyword
#### Vedi slides Ing. Ghironi

In [14]:
for key in tqdm(keywords_cosmetics):
    df[key] = df.title.str.count(key).fillna(0) + df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)
    
for key in tqdm(keywords_other):
    df[key] = df.title.str.count(key).fillna(0) #+ df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)

  0%|          | 0/32 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [15]:
df["total_cosmetics"] = 0
for key in keywords_cosmetics:
    if key == "body" or key == "lip(s)?": continue
    df["total_cosmetics"] += df[key]
    
df["total_other"] = 0
for key in keywords_other:
    df["total_other"] += df[key]

In [16]:
keywords_cosmetics

['cosmetic',
 'beauty',
 'body',
 'skin',
 'nail',
 'fingernail',
 'cleans(e|ing)',
 'manicur(e|ing)',
 'pedicur(e|ing)',
 'varnish',
 'polish',
 'nail(s| |-)*polish',
 'lip(s)?',
 'lip( |-)?stic( |k)+',
 'blush',
 'mascara',
 'make(s| |-)*up',
 'cream',
 'soap',
 'toothpaste',
 'sunscreen',
 'lotion',
 'deodorant',
 'care',
 'lip(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'nail(s| |-)*(care|polish)',
 'baby(s| |-)*(care|polish)',
 'anti(-)?ag(e|ing)',
 'colour|color',
 'finger']

##### cosmetic

In [17]:
keyword = keywords_cosmetics[0]
keyword

'cosmetic'

In [18]:
df[df[keyword] > 0][["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
1827,in vitro toxicogenomics for toxicity prediction using probabilistic component modeling and a compound-induced transcriptional response pattern,"C12Q1/68, C12Q1/6876, G06F19/12, G06F19/20, G16B20/00, G16B25/00, G16B5/00, G16C20/10, G16C20/30, G16C20/70","a novel method to predict toxicity and dose-dependent effects of an agent based on transcriptomic data analysis, by determining a predictive toxicogenomics space (ptgs) score. the ptgs score helps to predict and model the toxicity of compounds typically consisting of chemicals, pharmaceuticals, ...","1. a method of defining a predictive toxicogenomics space, ptgs, score indicative of a toxicity prediction for an agent, said method comprising: transforming gene data defining, for each instance of a plurality of instances, differential gene expression of genes in a test biological sample induc..."
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","a system for dispensing at least one makeup product, including a cup and a dispenser for filling the cup from the bottom with at least one product, the cup being secured to the dispenser at least while it is being filled with said product. the method for dispensing and evaluating makeup, includi...","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
5009,automated characterization and classification of microoganisms,"G01N21/65, G01N33/569","the present invention relates to an instrument, method, use and software program to obtain information rapidly about microorganisms that may spread uncontrolled in hospitals, water supply, food or when used in bio terrorism. vibrational spectroscopy provides data to a computer linked to one or m...","1. an instrument comprising a vibrational spectrometer, a processing means, a first spectral database [10] and a first information database [200], for obtaining information about a microorganism in a sample of interest by establishing vibrational spectroscopic similarity of the microorganism in ..."
6806,methods and systems for dynamic content provisioning,"G06F16/435, G06F16/438, G06F16/483, G06Q50/00","a method, system, and computer program product for dynamic content provisioning, such as for online marketing and product promotion, may include collecting user content from sources based upon collection parameters, and organizing the collected user content for optimization. there may also be th...","1. a system for dynamic content provisioning, comprising: at least one non-transitory memory configured to store computer program code instructions; and at least one processor configured to execute the computer program code instructions to: collect user content from one or more sources based on ..."
7363,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
7691,systems and methods for measuring and categorizing colors and spectra of surfaces,"A45D44/00, A61B5/00, G01J3/02, G01J3/46","a color categorization system and method is described. the method comprises the steps of taking at least one color or spectral measurem

In [19]:
#escludiamo tutti i sistemi per "raccomandare un prodotto"

df_candidate = df[(df[keyword] > 0) 
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("A01N")==0) #SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS, DETAILS, OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS, IN GENERAL
                  & (df.ipc_classes.str.count("(^| )A")>0)].query("title not in @df_selected")

df_candidate[["filename", "title", "ipc_classes","claims"]] 

,filename,title,ipc_classes,claims
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
106389,US2012003728_A1.txt,scalable portable sensory and yield expert system for biomass monitoring and production,"A01G31/02, C12M1/34, F21V33/00, G02F1/01","1. a system for optimizing plant growth in a liquid environment within an enclosed growth chamber, comprising: illumination means for controlling spatial, temporal and spectral characteristics of illumination surrounding the growth chamber, said illumination means comprising an array of individu..."
120991,EP3769852_A1.txt,"pump- actuated dispenser with integrated switch, system and method for remote monitoring of product consumption from said pump- actuated dispenser","A47K5/12, B05B11/00, B05B12/00, B05B15/30, H02N2/18","1. a pump-actuated dispenser (1) comprising: a compression pump (2) and a pump actuator (3) to operate the compression pump (2), the pump actuator (3) having a nozzle (4) for the output of a product to be dispensed, wherein the pump actuator (3) is displaceable between two end positions, a resti..."
166422,US2020405037_A1.txt,cosmetic package including a liquid reservoir,A45D34/04,"the embodiments of the invention in which an exclusive property or privilege is claimed are defined as follows:1. a cosmetic formula container, comprising: a container body configured to retain a portion of a cosmetic formula within a cavity, the container body having a neck configured to receiv..."
182147,US2020329742_A1.txt,"moisture control apparatus, moisture control method, program, storage medium, produced object, product, apparatus, and facility","A23L3/26, A23L3/30, A23L3/32, A23L5/10, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12, C02F1/48","1. a moisture control or current comprising: at least one electrode configured to generate at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and ultrasonic waves, wherein a predetermined voltage or current including a dc component ..."
210149,US2020399558_A1.txt,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


In [20]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

36

##### beauty

In [21]:
keyword = keywords_cosmetics[1]
keyword

'beauty'

In [22]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("C12Q")==0) #MEASURING OR TESTING PROCESSES INVOLVING ENZYMES, NUCLEIC ACIDS ...
                  & (df.ipc_classes.str.count("H04")==0) #ELECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")

df_candidate[["filename", "title", "ipc_classes","abstract", "claims"]]

,filename,title,ipc_classes,abstract,claims
5337,US2016369204_A1.txt,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
7363,WO2019143922_A1.txt,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","provided are a moisture control apparatus, a moisture control method, a program, a storage medium, a produced object, a product, an apparatus, and a facility that can improve the characteristics of an object through moisture control. in a moisture control apparatus according to one aspect of the...","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
31462,WO2019122838_A1.txt,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04",a beauty product device is disclosed which includes a variety of sensors for sensing the way in which a user is operating the device. the sensor data can be processed and compared with pre-stored data representative of a desired style and feedback messages generated and provided back to the user...,"1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
62406,US2019224871_A1.txt,personal appliance,"B26B19/38, B26B21/40, B26B21/52",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is associated with the handle. the implement displacement sensor measures a displacement of t...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connected to the implement connecting structure; c. an implement displacement sensor associated with the handle, the implement displacement sensor measuring a displacement of the impleme..."
140736,US2020171687_A1.txt,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connect

In [23]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

43

##### body (troppo rumore)

In [24]:
keyword = keywords_cosmetics[2]
keyword

'body'

In [25]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("A63H")==0) #TOYS, e.g. TOPS, DOLLS, HOOPS OR BUILDING BLOCKS
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("G10")==0) #MUSICAL INSTRUMENTS; ACOUSTICS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                  ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
566,systems and methods for data-driven movement skill training,"A63B24/00, A63B69/40, A63B71/06",a data-driven movement skill training system is disclosed. the system uses movement skill assessment and diagnostics at distinct levels of the human movement system hierarchy to specify training goals for a user. the system may provide various augmentations that are synthesized to help the user ...,"1. an apparatus for movement skill training, the apparatus comprising: a sensor system comprising one or more sensors configured to obtain movement data for a subject performing an activity; a processor system in communication with the one or more sensors, the processor system having a microproc..."
2654,mobile robot and control method therefor,"A47L9/28, B25J11/00, B25J19/02, B25J9/00, B25J9/16, G05D1/02",a mobile robot in one general aspect of the present invention includes: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the...,1. a mobile robot comprising: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the movement of the main body; a storage conf...
3478,connected food preparation system and method of use,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/08, H04W4/80, H05B1/02","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","we claim:1. a connected oven, comprising: an oven body defining a cooking cavity, the oven body comprising an oven base opposing an oven top, the cooking cavity comprising a set of low reflectance surfaces; a set of optical sensors cooperatively defining a field of view, the set of optical senso..."
4902,dribbling and ball control sports training system and method,A63B69/00,"a sports training method is described comprising the following steps: classifying the dribbling and ball or puck control motions into modular predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and b...","1. a sports training method which comprises the following steps: classifying different dribbling and ball or puck control motions into predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and ball ..."
5260,a connected oven,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/06, F24C7/08, H04W4/80, H05B1/02, H05B6/68","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","1. a connected oven (100), comprising: • an oven body defining a cooking cavity (200), the oven body comprising an oven base (250) and an oven top (230); • an oven door (210) mounted to the oven body, the oven door (210) comprising: • a transparent window coextensive with the cooking cavity (200..."
8076,multiview body camera system with environmental sensors and alert features,"A41D1/00, A41D1/04, A42B3/04, G02B27/01, H04N5/00, H04N5/225, H04N5/247","a multiview body camera system with a vest, a helmet, shoulder pads, camera units providing video, directional microphones providing audio, sensors providing motion detection and other functions is described herein. a control module controls camera units, microphones, sensors and all other circu...",i claim:1. an enhanced situation

##### skin (da provare combinazioni)

In [26]:
keyword = keywords_cosmetics[3]
keyword

'skin'

In [27]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
538,ligands for odor receptors and olfactory neurons,"A01M1/10, A01N33/00, A23L27/10",the disclosure provides compounds useful as insect repellents and compositions comprising such repellents. the disclosure further provides insect traps and method for identifying ligands and cognates for biological molecules.,"1-41. (canceled)42. a composition, comprising one or more compounds selected from table 9, table 10, and any combination thereof, in a trap, a cream, a lotion, a spray, a dust, a vapor emitter, a candle, an oil, a wicked apparatus, a fan, a vaporizer, a perfume, a cologne, a fragrance, a deodora..."
2533,rowing exercise machines having a configurable rowing feel,"A63B21/00, A63B21/005, A63B21/22, A63B22/00, A63B24/00, A63B71/06","among other things, a rowing exercise machine includes a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the ...","1. a rowing exercise machine comprising a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the rowing grip dur..."
4603,creating music by concatenative synthesis,"A63H5/00, G04B13/00, G10H7/00, H03G5/00","automated creation of new music by listening is disclosed. a method to create new music may comprise listening to a plurality of music, learning from the plurality of music, and performing concatenative synthesis based on the listening and the learning to create the new music. the method may be ...",it is claimed:1. a method to create new music comprising: listening to a plurality of music learning from the plurality of music performing concatenative synthesis based on the listening and the learning to create the new music.2. the method of claim 1 wherein the music comprises at least one of...
11876,personally powered appliance,"A43B3/00, G08B21/02, H01L41/113, H04B1/38, H04Q9/00","systems and methods are disclosed for powering a processor and a wireless transceiver in a sole having a sole plate and one or more spring elements disposed in a forefoot area of the sole plate, each of said spring elements including a piezoelectric generator coupled thereto, wherein each of the...","1. an apparatus, comprising: a processor; a wireless transceiver coupled to the processor; a power supply coupled to the processor and wireless transceiver located proximal to a sole, said sole including: a sole plate; and one or more leaf spring elements extending at an angle from the sole plat..."
12941,devices for prevention of pressure ulcers,"A47C27/00, A47C27/08","a support surface, such as an underpad, for preventing pressure sores in a patient is a multi-layer textile assembly having a skin contact layer formed from a fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a thickness of less than ...","1. a support surface for preventing pressure ulcers in a patient, the support surface comprising a multi-layer composite fabric comprising: a skin contact layer comprising a woven fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a th..."
15513,computer-implemented question and answer game,A63F9/24,"a computer-implemented question and answer game based on using opinion polls that have already been carried out on a representative sample of people. results of the opinion polls and predefined questions are stored in a knowledge base associated with the system, which is typically implemented on...","1. a method of allocating a prize to at least one individual using a computer system having at least one host computer terminal for presenting and collecting -data, a question engine for sendi

In [28]:
df_selected = pd.concat([df_selected, df[df.title == "personal care device with camera"]])
len(df_selected)

45

##### skin (da provare combinazioni)

In [29]:
keyword = keywords_cosmetics[4]
keyword

'nail'

In [30]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  & (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  & (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
10744,"improved microlayer membranes, mproved battery separators, and related methods","B29C48/00, B29C48/08, B29C48/21, B32B27/08, B32B27/32, H01M2/14, H01M2/16","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes of the same thickness. the preferred multilayer microporous membrane comprises...","1. a battery separator for a lithium battery comprising: at least one microporous separator membrane or sub-membrane comprising a plurality of porous or microporous polymer microlayers or nanolayers, wherein at least one of the individual microlayers or nanolayers comprises a different or distin..."
14944,power center assembly having electrical connection-protection and optional detachable surface mount,"H01R13/516, H01R13/66, H01R13/70, H01R25/00, H02G3/18","a power center assembly having a hooded member for protecting an electrical connection between a peripheral device plug and its respective outlet and a mount member for detachably mounting the power center to a surface. in particular, a power center assembly is provided having a member for prote...","1. a power center assembly, comprising: a power strip, said power strip having a housing member and at least one outlet for powering at least one peripheral device, said housing member having a bottom surface; and mechanical means for preventing inadvertent mechanical and electrical disconnec..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
22515,system and methods for patient data and treatment management,,"described are systems and methods for the analysis of information, particularly information obtained from the mass spectroscopic (proteomic) analysis of biological samples and associated patient and clinical information. the systems and methods of the invention may be used for a variety of purpo...","what is-1cealmeo fs:11. a system, comprising: a system input to receive a quantity of proteomic data obtained from mass spectrometric analysis of a biological sample; a metadata repository to store a quantity of metadata that includes the quantity of proteomic data; a rules database to store at ..."
27088,interchangeable footwear system and method,"A43B21/36, A43B3/24","a footwear system (s) and corresponding methods (m1) and (m2) for providing a plethora of footwear fashion options via a plurality of interchangeable components, the interchangeability of which being facilitated by structurally stable quick-coupling and quick-decoupling techniques. the system (s...","1. an interchangeable footwear system, comprising: a plurality of interchangeable uppers; a plurality of interchangeable heels; and a quick-release device comprising a rigid slide reversibly fastenable to the plurality of interchangeable uppers and reversibly fastenable to the plurality of inter..."
40165,"improved microlayer membranes, improved battery separators, and related methods","B29C47/06, B32B27/00, H01M10/052, H01M2/14, H01M2/16, H01M2/18","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes 

In [31]:
df_selected = pd.concat([df_selected, df[df.title == "automatic nail polish application system and method"]]) 
len(df_selected)

46

##### fingernail

In [32]:
keyword = keywords_cosmetics[5]
keyword

'fingernail'

In [33]:
df_candidate = df[(df[keyword] > 0)
                  
                 
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                  
                   
                  
                  
                 ].query("title not in @df_selected")


df_candidate[["title", "ipc_classes","abstract", "claims"]] #Contiene solo metodi relativi al miglioramento dei dispositivi touch

,title,ipc_classes,abstract,claims


In [34]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

46

##### cleans(e|ing)

In [35]:
keyword = keywords_cosmetics[6]
keyword

'cleans(e|ing)'

In [36]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  & (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  & (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                  & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B66")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A46")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                  
                  
                  
                  
                  
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [37]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

46

##### manicur(e|ing)

In [38]:
keyword = keywords_cosmetics[7]
keyword

'manicur(e|ing)'

In [39]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."


In [40]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

47

##### pedicur(e|ing)

In [41]:
keyword = keywords_cosmetics[8]
keyword

'pedicur(e|ing)'

In [42]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [43]:
keyword = keywords_cosmetics[11]
keyword

'nail(s| |-)*polish'

In [44]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the invention proposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising: - acquiring an image of the body area without makeup of a subject, - determining first color parameters of the pixels of the image corresponding to the body area without makeup,...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
158644,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the inventionproposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising:∘ acquiring an image of the body area without makeup of a subject,∘ determining first color parameters of the pixels of the image corresponding to the body area without makeup,∘ i...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (11), and comprising : ∘ acquiring (210) an image of the body area without makeup of a subject, ∘ determining (230) first color parameters of the pixels of the image corresp..."


In [45]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

50

In [46]:
keyword = keywords_cosmetics[12]
keyword

'lip(s)?'

In [47]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G08")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A47")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A43")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B64")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A44")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B31")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B66")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B29")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B26")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 & (df.ipc_classes.str.count("B22")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B65")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A22")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B65")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A45")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B82")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B42")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                  
                  
                  
                  
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
3916,"leveraging genomic, phenotypic and pharmacological data to cure disease",,"the present invention provides a process and method for repurposing existing compounds by leveraging genomic, phenotypic and pharmacological data to cure disease. applying advanced mathematical analytics using massively interconnected computing capabilities to identify target rich sets of existi...","1. a method of identifying a compound that modulates at least one of a group of diseases, said method comprising: a. selecting a symptom or disease; b. feeding said symptom or disease into a search field of a structured database; c. querying said at least one structured database for content of o..."
23420,virtual reality and mixed reality enhanced elliptical exercise trainer,,"a system for a virtual and mixed-reality enhanced elliptical exercise trainer that allows a user to exercise while interacting with a virtual reality or mixed reality environment using whole body movements and positioning, and receiving haptic feedback associated with the user's interaction with...","1. a virtual reality and mixed reality enhanced elliptical exercise trainer, comprising:an elliptical exercise trainer; anda body joystick comprising:at least one rigid, semi-rigid, or flexible support arm affixed to the elliptical exercise trainer, another object, or a fixed surface;at least on..."
24892,methods and systems for profiling personalized biomarker expression perturbations,,"the disclosed methods and systems allow for a systematic quantification of the heterogeneity of disease states between different subjects on a molecular (e.g., gene or protein expression) level. one example embodiment of the invention is a method for determining a disease state of a patient. the...","1. 1-5. (canceled)6. a method of classifying the disease state of a subject, the method comprising: detecting differential expression of one or more analytes or genes in a subject to obtain a personalized perturbation profile for the subject; and determining the fraction of analytes or genes fro..."
40097,a subject-specific system and method for prevention of body adaptation for chronic treatment of disease,,"there are provided herein a system and a computer implemented method for preventing, mitigating or treating partial/complete loss of effect of one or more drugs or medical devices administered to or used by a subject in need thereof due to adaptation, tolerance, and/or tachyphylaxis, and/or for ...","what we claim is:1. a computer implemented method for preventing, mitigating or treating partial/complete loss of effect of one or more drugs or medical devices administered to or used by a subject in need thereof due to adaptation, tolerance, and/or tachyphylaxis, and/or for preventing, mitigat..."
50567,hierarchical optimized detection of relatives,,a system for evaluating a dna sample and determining whether the sample contains related individuals and/or unrelated individuals with high levels of alleles sharing. trained and pre-validated machine learning algorithms are to rapidly and probabilistically assess the presence of relatives in a ...,"1. a method of determining whether a sample of dna contains related contributors, comprising the steps of: receiving peak detection signals for a predetermined number of loci in a dna sample; establishing from the peak detection signals a first set of peak detection data to be transformed and a ..."
55908,optimizing successful outcomes for repurposing existing pharmaceutical compounds,,"the present invention provides a process and method for preventing and/or treating protein based diseases, including, but not limited to: alzheimer's and other amyloid based disease groups. this method recognizes correlative similarities across genomics, phenotypic and pharmacologic analytics an...",1. an improved method for repurposing a known compound for use in treating a disease for which said compound was not clinically prescribed compris

In [48]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

66

In [49]:
keyword = keywords_cosmetics[13]
keyword

'lip( |-)?stic( |k)+'

In [50]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
61057,method and composition for removing waxy soils,"C11D11/00, C11D17/04, C11D3/18, C11D3/43, C11D7/24, C11D7/26","the invention generally pertains to a method and composition for removing waxy soils. the invention also pertains to a method and composition for removing lip cosmetic soils such as lipsticks, chapsticks(tm), and the like. the invention further pertains to a method and composition for removing l...",1. (canceled)2. (canceled)3. (canceled)4. (canceled)5. a method of removing a lip cosmetic soil from drinkware comprising: a. providing drinkware having a lip cosmetic soil; b. placing the drinkware in a pretreating apparatus having a pretreating composition; c. applying a pretreating composi...
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the invention proposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising: - acquiring an image of the body area without makeup of a subject, - determining first color parameters of the pixels of the image corresponding to the body area without makeup,...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
130055,composition comprising mannose oligosaccharide and process for making same and use thereof,"A23K20/163, A23K50/30, A23K50/60, A23K50/75, A23L33/00, A23L33/125, A61K31/702, A61K8/60, A61Q1/00, A61Q11/00, A61Q19/00",the present invention relates to a composition comprising mannose oligosaccharide and a process for making such. the present invention further relates to a composition comprising mannose oligosaccharide for use in the treatment of a human or animal.,"1. a mannose oligosaccharide composition, characterized in that the composition is soluble in water.2. the composition of claim 1 characterized in that it comprises less than 0.1 wt % db of beta-glucan.3. the composition of claim 1 characterized in that a mannose oligosaccharide content that is ..."
158644,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the inventionproposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising:∘ acquiring an image of the body area without makeup of a subject,∘ determining first color parameters of the pixels of the image corresponding to the body area without makeup,∘ i...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (11), and comprising : ∘ acquiring (210) an image of the body area without makeup of a subject, ∘ determining (230) first color parameters of the pixels of the image corresp..."
209315,determining age ranges of skin samples,"A61K31/56, A61K38/16, A61K38/43, A61K8/18, A61Q17/04, C12Q1/68, C40B30/04, C40B40/06","the present invention provides methods for characterizing a skin sample of a subject as belonging to an age range by analyzing nucleic acid or protein molecules obtained from the subject. the methods include analyzing expression or mutations in epidermal samples, of one or more skin markers. the...","1. a method for determining the age range of a skin sample of a subject comprising analyzing nucleic acid from one or more genes listed in table 1, table 2, table 3, or any combination thereof, in a sample of the skin from the subject, thereby determining the age range of the skin of the subject..."


In [51]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

71

In [52]:
keyword = keywords_cosmetics[14]
keyword

'blush'

In [53]:
df_candidate = df[(df[keyword] > 0)
                    & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [54]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

71

In [55]:
keyword = keywords_cosmetics[15]
keyword

'mascara'

In [56]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06T19/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G06F17/30")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06K9/62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06F17/30")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 
                  
                  
                
                
                  
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
8936,automated imaging system for evaluating the curl of a keratinous substrate,"G06T7/11, G06T7/50, G06T7/73, G06T7/90","a system is provided for evaluating a curl of at least one eyelash sample that includes a plurality of fibers, comprising: a platform configured to receive a sample of the eyelash sample; an image capturing device configured to capture profile image of the eyelash ample; and a processing system ...","1. a system for evaluating a curl of at least one eyelash sample that includes a plurality of fibers, comprising: a platform configured to receive a sample of the eyelash sample; an image capturing device configured to capture profile image of the eyelash sample; and a processing system configur..."
116454,body image templates with pre-applied beauty products,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","disclosed methods and systems relate to enabling a beauty analysis. one of the methods involves facilitating display, on a display device, of a plurality of templates, wherein at least some of the templates are representative of a portion of a face having a simulation of use of a beauty product....","1. a method of enabling a beauty analysis, the method comprising: facilitating display, on a display device, of a plurality of templates, wherein at least some of the templates are representative of a portion of a face having a simulation of use of a beauty product; enabling selection of at leas..."
163843,identification and presentation of analogous beauty case histories,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","methods and systems are provided for assisting a subject in selecting a beauty product. in accordance with an embodiment, a method may include maintaining a plurality of sets of time-lapse body images for a plurality of individuals. in addition, an identity of at least one beauty product used by...","1. a method of assisting a subject in selecting at least one beauty product, the method comprising: maintaining a plurality of sets of time-lapse body images for a plurality of individuals; associating with each of the plurality of sets of time-lapse body images an identity of at least one beaut..."
210373,simulation of an aesthetic feature on a facial image,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","disclosed methods and systems enable simulated use of an aesthetic feature on a simulated facial image. one of the methods involves enabling an individual to construct a simulated facial image using a facial construction computer program, wherein the facial construction computer program permits ...","1. a method of enabling simulated use of an aesthetic feature on a simulated facial image, the method comprising: enabling an individual to construct a simulated facial image using a facial construction computer program, the facial construction computer program permitting the individual to selec..."


In [57]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

75

In [58]:
keyword = keywords_cosmetics[16]
keyword

'make(s| |-)*up'

In [59]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
40462,resolving opaqueness of complex machine learning applications,,computing systems and technical methods that transform data structures and pierce opacity difficulties associated with complex machine learning modules are disclosed. advances include a framework and techniques that include: i) global diagnostics; ii) locally interpretable models lime-sup-r and ...,1. a computing system that pierces a black box effect of a complex machine learning module comprising: a replicator module that employs local and global effect modeling on a plurality of machine learning modules that include at least one complex machine learning module wherein the local and glob...
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the invention proposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising: - acquiring an image of the body area without makeup of a subject, - determining first color parameters of the pixels of the image corresponding to the body area without makeup,...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
85479,instantaneous price quotation system for custom manufactured parts,,an automated quotation system is provided for generating firm binding price quotations for custom manufactured parts in either an online or offline context.,"1. a method of providing a firm price quotation for a custom manufactured part, comprising: (a) receiving access on a server computer system from a customer computer over a global communication network; (b) uploading from the customer computer to the server computer system a pre-existing compute..."
158644,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the inventionproposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising:∘ acquiring an image of the body area without makeup of a subject,∘ determining first color parameters of the pixels of the image corresponding to the body area without makeup,∘ i...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (11), and comprising : ∘ acquiring (210) an image of the body area without makeup of a subject, ∘ determining (230) first color parameters of the pixels of the image corresp..."


In [60]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

79

In [61]:
keyword = keywords_cosmetics[17]
keyword

'cream'

In [62]:
df_candidate = df[(df[keyword] > 0)
                  
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A42")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
3916,"leveraging genomic, phenotypic and pharmacological data to cure disease",,"the present invention provides a process and method for repurposing existing compounds by leveraging genomic, phenotypic and pharmacological data to cure disease. applying advanced mathematical analytics using massively interconnected computing capabilities to identify target rich sets of existi...","1. a method of identifying a compound that modulates at least one of a group of diseases, said method comprising: a. selecting a symptom or disease; b. feeding said symptom or disease into a search field of a structured database; c. querying said at least one structured database for content of o..."
15551,method and apparatus for enhancing nervous function,"A61B5/00, A61B5/01, A61B5/0205, A61B5/024, A61B5/04, A61B5/0402, A61B5/0476, A61B5/0488, A61B5/053, A61B5/08, A61B5/145","disclosed is a method for interacting with the nervous system. the method includes detecting signals associated with a biological function at one or more sensors. it also includes processing the signals to create a representation thereof, delivering effector responses based on the representation...","1. a method for interacting with the nervous system, the method comprising: detecting signals of normal and abnormal function of a body function of an individual at one or more sensors, without requiring knowledge of the precise neurophysiological mapping of the signals; mathematically processin..."
18306,determining age ranges of skin samples,"A61K31/56, A61K38/16, A61K38/43, A61K8/18, A61Q17/04, C12Q1/68, C40B30/04, C40B40/06","the present invention provides methods for characterizing a skin sample of a subject as belonging to an age range by analyzing nucleic acid or protein molecules obtained from the subject. the methods include analyzing expression or mutations in epidermal samples, of one or more skin markers. the...","1. a method for determining the age range of a skin sample of a subject comprising analyzing nucleic acid from one or more genes listed in table 1, table 2, table 3, or any combination thereof, in a sample of the skin from the subject, thereby determining the age range of the skin of the subject..."
22700,method and system for combining physiological and machine information to enhance function,"A61B5/00, A61B5/0205, A61B5/04, A61B5/0408, A61B5/0488, A61B5/0492",the present invention relates generally and specifically to combining biological sensors with external machines using machine learning to form computerized representations that can control effectors to deliver therapy or enhance performance.,"1. a method for interacting with the human body, the method comprising: detecting bodily signals associated with one or more bodily functions at one or more sensors associated with the human body; processing the bodily signals to create one or more sensed signatures of the one of more bodily fun..."
31251,live biotherapeutic compositions and methods,A61K39/02,"live biotherapeutic compositions and methods are provided for treatment, prevention, or prevention of recurrence of skin and soft tissue infections, such as mastitis and/or intramammary infections, for example, in cows, goats, sows, and sheep. methods include decolonizing and durably replacing w...","1. a live biotherapeutic composition for treatment or prevention of bovine, caprine, ovine, or porcine mastitis and/or intramammary infection comprising at least one synthetic microorganism, and a pharmaceutically acceptable carrier, wherein the synthetic microorganism comprises a recombinant nu..."
34424,systems and methods involving non-surgical laser treatment of a fibrous mass,"A61B8/08, A61N5/067","systems and methods for treating fibrous masses associated with conditions such as morton's neuroma, stump neuroma, plantar fibroma, plantar fibromatosis, plantar fasciitis, plantar fasciopathy, and achilles tendinopathy. according to implementation herein, exemplary methods may c

In [63]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

96

In [64]:
keyword = keywords_cosmetics[18]
keyword

'soap'

In [65]:
df_candidate = df[(df[keyword] > 0)
                  
                 & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 & (df.ipc_classes.str.count("A47")==0) #Escludo "COMPUTING; CALCULATING; COUNTING" 
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
32332,hand hygiene compliance system,"B67D7/06, G08B1/08, G08B23/00","a system and method of encouraging compliance of hand hygiene in an environment where users move from zone to zone and are required to perform hand hygiene between the zones. users carry a wearable zone sensor which detects zones, detects hand hygiene actions, logs time of changing zones, and ha...","1-17. (canceled)18. a system for monitoring hand hygiene among a number of human attendants in a facility, comprising: a. a number of wearable monitor units, each monitoring unit to be worn by one attendant, and each including: i. a data transfer portion operable in a first phase for receiving s..."
93679,hand hygiene compliance system,"A61G12/00, G07C1/10, G08B21/00","a system and method of encouraging compliance of hand hygiene in an environment where users move from zone to zone and are required to perform hand hygiene between the zones. users carry a wearable zone sensor which detects zones, detects hand hygiene actions, logs time of changing zones, and ha...","we claim: 1 a wearable monitoring unit comprising a a data transfer portion operable for receiving sensory data reporting a hygiene event from a hygiene detector, and zone data from a zone beacon, the zone data including zone location data and/or zone type data, b a memory portion including i a..."
114237,hand hygiene compliance system,"B67D7/06, G08B1/08, G08B23/00","a system and method of encouraging compliance of hand hygiene in an environment where users move from zone to zone and are required to perform hand hygiene between the zones. users carry a wearable zone sensor which detects zones, detects hand hygiene actions, logs time of changing zones, and ha...","1. a wearable hygiene monitoring unit for communicating with a plurality of zone beacons each transmitting respective zone identifying data, the unit comprising: a receiver; an alerting unit; a memory having stored therein a group of hygiene status subroutines; and a processor operable to receiv..."
131174,display system for usage compliance of hygiene equipment,G08B21/24,"displaying an indicator of a usage of hygiene equipment by one or more operators, the system including a first receiving section configured to receive usage data from an equipment sensor arrangement, said usage data indicating a usage of said hygiene equipment; a calculation section configured t...","1. a system for displaying an indicator of a usage of hygiene equipment by one or more operators, the system comprising: a first receiving section configured to receive usage data from an equipment sensor arrangement, said usage data indicating a usage of said hygiene equipment; a calculation se..."
161871,reward hygiene system,G08B21/24,"a monitoring system for hygiene equipment, the system comprising an interface configured to receive information indicating an instance of using a piece of hygiene equipment; a determination section configured to determine whether the instance is a compliant instance to use the piece of hygiene e...","1. a monitoring system for hygiene equipment, the system comprising : an interface configured to receive information indicating an instance of using a piece of hygiene equipment ;a determination section configured to determine whether the instance is a compliant instance to use the piece of hygi..."
191501,method and system for improving infection control in a facility,"G08B21/24, G08B29/18",methods and systems for monitoring procedural compliance of staff in a facility. a system may include a plurality of sensors positioned adjacent a hand washing station. each of the plurality of sensors may be configured to provide a corresponding sensor output signal that is indicative of whethe...,"1. a system for monitoring hand-hygiene of staff in a facility, comprising: a plurality of sensors positioned adjacent a hand washing station, wherein each of the plurality of sensors is free from imaging pixels that form a visua

In [66]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

102

In [67]:
keyword = keywords_cosmetics[19]
keyword

'toothpaste'

In [68]:
df_candidate = df[(df[keyword] > 0)
                  
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
65191,oral hygiene system for compliance monitoring and tele-dentistry system,"A46B15/00, A61C17/16, A61C17/22","oral hygiene monitoring system is disclosed to track motion and orientation of an oral hygiene device. the control system may process data output from a motion sensor to determine position and orientation of an oral hygiene device with respect to a user's mouth. in some examples, the system may ...",1. an oral hygiene system for monitoring compliance with an oral hygiene regimen comprising:an oral hygiene device comprising a handle and a head;an optical sensor;a memory containing machine readable medium comprising machine executable code having stored thereon instructions for performing a m...


In [69]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

103

In [70]:
keyword = keywords_cosmetics[20]
keyword

'sunscreen'

In [71]:
df_candidate = df[(df[keyword] > 0)
                  
                 & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
59490,method and system for managing and quantifying sun exposure,"A61B5/00, A61B5/103","embodiments of the present invention disclose a method and system for managing pre- and in-sun exposure of skin. the method comprises during pre-sun exposure phase, inputting a location information of a user, and other contextual ambient information therefor, thereby facilitating determination o...","1. a method for managing sun exposure of skin comprising: in a pre-sun exposure phase, inputting the location information of a user, and other contextual ambient information therefor, thereby facilitating determination of the uv level thereof; determining a baseline information in connection wit..."
88971,methods and compositions for identification and modulation of microbiome biochemical pathways to alter phenotype,"A61K31/015, A61K31/03, A61K31/11, A61K31/20, A61K9/00","provided, in part, are methods, compositions, and systems are provided for detecting one or more repellent-generating enzymatic pathways by characterizing the skin microbiomes and metabolomes of populations of individuals both naturally resistant and highly prone to bites from blood sucking arth...","1. 1-50. (canceled)51. a method of repelling insects, comprising: topically applying on a subject a composition that modulates a skin microbiome biochemical pathway; and modulating the skin microbiome biochemical pathway to generate a compound that repels insects.52. the method of claim 51, wher..."


In [72]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

105

In [73]:
keyword = keywords_cosmetics[21]
keyword

'lotion'

In [74]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B82")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
31251,live biotherapeutic compositions and methods,A61K39/02,"live biotherapeutic compositions and methods are provided for treatment, prevention, or prevention of recurrence of skin and soft tissue infections, such as mastitis and/or intramammary infections, for example, in cows, goats, sows, and sheep. methods include decolonizing and durably replacing w...","1. a live biotherapeutic composition for treatment or prevention of bovine, caprine, ovine, or porcine mastitis and/or intramammary infection comprising at least one synthetic microorganism, and a pharmaceutically acceptable carrier, wherein the synthetic microorganism comprises a recombinant nu..."
34424,systems and methods involving non-surgical laser treatment of a fibrous mass,"A61B8/08, A61N5/067","systems and methods for treating fibrous masses associated with conditions such as morton's neuroma, stump neuroma, plantar fibroma, plantar fibromatosis, plantar fasciitis, plantar fasciopathy, and achilles tendinopathy. according to implementation herein, exemplary methods may comprise non-sur...","1. a method for treating a fibrous mass, comprising:identifying a location of the fibrous mass; andnon-surgically delivering electromagnetic energy having a spot size of about 8-10 mm, and other specified parameters, to the fibrous mass.2. the method of claim 1 or the invention of any claim here..."
71206,ligands for odor receptors and olfactory neurons,"A23L27/10, C40B30/04, G01N33/68",the disclosure provides compounds useful as insect repellents and compositions comprising such repellents. the disclosure further provides insect traps and method for identifying ligands and cognates for biological molecules.,"1. a method of identifying a ligand for a biological molecule comprising: (a) identifying a known ligand or set of known ligands for a biological molecule, or identifying a compound which causes a specific biological activity, (b) identifying a plurality of descriptors for the known ligand or ..."
77413,"topical/on-the-skin estrogen cream, lotion, ointment, hair tonic, shampoo and wound healing powder",A61K8/64,"the claimed invention is the topical-to-skin use of generic estradiol, estriol, conjugated estrogens, or any other synthetic estrogen within a non-absorbable generic vehicle of cream, lotion, ointment, shampoo, hair tonic, and wound-healing powder, having no absorbents or deconstructors in their...","1. the problem: the natural aging of the skin, the body's largest organ, becomes apparent from the mid-30's to early 40's and beyond. the normal skin aging physiology evidences as skin gradually becoming thinner, dryer, more wrinkled, more easily bruised, injured and infected, and susceptible to..."
88971,methods and compositions for identification and modulation of microbiome biochemical pathways to alter phenotype,"A61K31/015, A61K31/03, A61K31/11, A61K31/20, A61K9/00","provided, in part, are methods, compositions, and systems are provided for detecting one or more repellent-generating enzymatic pathways by characterizing the skin microbiomes and metabolomes of populations of individuals both naturally resistant and highly prone to bites from blood sucking arth...","1. 1-50. (canceled)51. a method of repelling insects, comprising: topically applying on a subject a composition that modulates a skin microbiome biochemical pathway; and modulating the skin microbiome biochemical pathway to generate a compound that repels insects.52. the method of claim 51, wher..."
91822,"methods for identifying arthropod repellents and attractants, and compounds and compositions identified by such methods","A61K49/00, G01N33/00",provided herein are screening methods for identifying compounds for use as an arthropod repellent based on the masking or inhibition of the detection of the skin odor by a cpa neuron. provided herein are also screening methods for identifying compounds for use as an arthropod attractant based on...,"attorney docket no. 67703

In [75]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

116

In [76]:
keyword = keywords_cosmetics[22]
keyword

'deodorant'

In [77]:
df_candidate = df[(df[keyword] > 0)
                    & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [78]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

116

In [79]:
keyword = keywords_cosmetics[23]
keyword

'care'

In [80]:
df_candidate = df[(df[keyword] > 0)
                   
                  & (df.ipc_classes.str.count("G09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"  
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G08")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING" 
                  & (df.ipc_classes.str.count("G07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"& 
                  & (df.ipc_classes.str.count("A63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A46")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                  
                  
                  
                  
                  
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
7363,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
8045,cancer evolution detection and diagnostic,,"the present disclosure provides methods for determining a probability that after any of a number of therapeutic interventions, an initial state of a subject, such as somatic cell mutational status of a subject with cancer, will develop a subsequent state. such probabilities can be used to inform...","1. a computer-implemented method, comprising: (a) obtaining information about a plurality of subjects with cancer at a first time point, wherein the information comprises, for each subject of the plurality of subjects, at least a genetic profile of a tumor obtained by genotyping nucleic acids fr..."
10794,system and method for automated diagnosis and treatment,,"various embodiments of the present application relate to systems and methods for automated diagnosis and treatment, especially for automated diagnosis and treatment integrated with artificial intelligence as well as incorporating treatment scheme feedback. the automated healthcare diagnosis and ...","we claim:1. a method for automated healthcare service, the method comprising:receiving one or more of diagnosis results generated by a diagnosis module, the generated one or more diagnosis results being related to one or more input symptoms from a patient via a patient interface;selecting one or..."
10994,method for treating antibody-mediated rejection post-transplantation,C07K16/28,"described herein are methods for treating antibody mediated rejection of transplanted organs using inhibitors of il-6. in one embodiment, the il-6 inhibitor is tocilizumab and is administered simultaneously or sequentially with intravenous immunoglobulin (ivig).","1. a method for treating, inhibiting and/or reducing the severity of antibody mediated rejection (abmr) of an organ transplant in a subject in need thereof comprising: providing an inhibitor of il-6; and administering an effective amount of the il-6 inhibitor to a subject in need thereof, wherei..."
13976,method and system for evaluating product research,,"systems and methods for evaluating product research are disclosed herein. a user inputs product information to a website or app, after which a database is searched for documentation related to the product. this documentation is scanned for the answers to questions related to the product. based o...","what is claimed is:1. a method for assessing quality of research, the method comprising:receiving data into a computing device;accessing a database, wherein the database contains documentation associated with the data;applying an algorithm to the database and thereby mining the database for prod..."
17629,patient condition detection and mortality,,"when prediction onset of a medical condition for a patient, multiple sour

In [81]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

151

In [82]:
keyword = keywords_cosmetics[24]
keyword

'lip(s| |-)*(care|polish)'

In [83]:
df_candidate = df[(df[keyword] > 0)
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [84]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

151

In [85]:
keyword = keywords_cosmetics[25]
keyword

'skin(s| |-)*(care|polish)'

In [86]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
22523,computer-aided diagnostic systems and methods for determining skin compositions based on traditional chinese medicinal (tcm) principles,"A61B5/00, A61B5/16","computer-aided systems and methods are provided for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles by statistically analyzing biological and/or psychological information collected from such user, such as age, gender, bodily sensati...","what we claim is:1. a computer-aided diagnosis system for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles, said system comprising: (1) a data entry module configured for entering into said system a clinical data set obtained from t..."
37260,customized skin care products and personal care products based on the analysis of skin flora,"C12Q1/689, G16B30/00, G16B40/00","embodiments of the present invention relate to a combination of experimental and computational workflows that allow characterization of skin and subcutaneous tissue microbial flora and its associated metabolome, aiming to first evaluate an individual's skin and subcutaneous tissue to determine i...","1. a method for characterizing an individual who provides a microbiome, comprising: (a) generating a sequence profile comprising a nucleotide sequence associated with said microbiome; (b) generating a metabolome profile selected from the group consisting of a cellular growth rate associated with..."
52149,skin analysis methods,"A61B6/00, G01N21/21","the present invention generally relates to a skin care device comprising an electromagnetic radiation source capable of directing incident electromagnetic radiation to a location on the skin of a user, a radiation detector for measuring various parameters of radiation re-emitted from the locatio...","what is claimed is:1. a skin care device, the device comprising: an electromagnetic radiation source capable of directing incident electromagnetic radiation to a location on the skin of a user; a radiation detector for measuring various parameters of radiation re-emitted from the location; and..."
55465,predicting skin age based on the analysis of skin flora and lifestyle data,"A61B10/02, A61B34/10, C12Q1/689, G16B20/00, G16B5/00, G16H20/30, G16H50/30",the present invention relates to a combination of experimental and computational workflows that allow characterization of specific molecular mechanisms by which the microbiome contribute to skin health and skin age.,"1. a method for determining a skin age of an individual and improving an overall skin quality of the individual, comprising: (a) receiving metadata related to the individual; (b) determining the skin age of the individual using a computer-implemented model that models relationships of population..."
57661,"system, device and method for dermal imaging","A61B5/00, A61B6/00, G01J4/00","in embodiments of the present invention, systems and methods of a non-invasive imaging device may comprise an illumination source comprising an incident light source to direct light upon skin, and a detector for detecting the degree of polarization of light reflected from the skin. a system and ...","what is claimed is:1. a non-invasive imaging device, comprising: an illumination source comprising an incident light source to direct light upon skin; and a detector for detecting the degree of polarization of light reflected from the skin.2. the device of claim 1, wherein the illumination sourc..."
72418,use of artificial intelligence to identify novel targets and methodologies for skin care treatment,G16H50/20,use of an artificial intelligence engine to provide skin care treatment is disclosed.,"1. a method of providing skin care treatment, comprising: receiving user-specific information; accessing a data structure containing information reflecting relationships between categories of user-specific information and skin care treatment in

In [87]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

165

In [88]:
keyword = keywords_cosmetics[26]
keyword

'skin(s| |-)*(care|polish)'

In [89]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("G06F19")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("G06K9")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A23K20")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A43B13")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
6036,computer-aided diagnostic systems and methods for determining skin compositions based on traditional chinese medicinal (tcm) principles,"G06F15/18, G06F17/00, G06N5/04, G06N7/02","computer-aided systems and methods are provided for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles by statistically analyzing biological and/or psychological information collected from such user, such as age, gender, bodily sensati...","1. a computer-aided diagnosis system for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles, said system comprising: (1) a data entry module configured for entering into said system a clinical data set obtained from the specific user, ..."
15374,method of generating user feedback information to enhance product use results,"A45D44/00, G06Q30/02, G06Q30/06","a skin care computing device that includes a user interface, a camera, a speaker, a communication interface, and a personal care assistant application. the personal care assistant application can identify personal care products being used by a user and product use event data describing the use o...","1. a computing device for providing feedback regarding skin care products, the computing device comprising: a user interface; a communication interface; one or more processors; and a non-transitory computer-readable memory coupled to the one or more processors, the user interface, and the commun..."
22357,machine implemented virtual health and beauty system,"A45D44/00, A61B5/00, G06F16/9535, G06N20/00",a system is provided that includes one or more memory circuits configured to store a database. the system further includes one or more processing circuits configured to: accept data from a user indicative of a health or beauty state thereof; and provide a regimen recommendation based on machine ...,1. a system comprising: one or more memory circuits configured to store a database; one or more processing circuits configured to: accept data from a user indicative of a health or beauty state thereof; and provide a regimen recommendation based on machine learning over the database and the acce...
22523,computer-aided diagnostic systems and methods for determining skin compositions based on traditional chinese medicinal (tcm) principles,"A61B5/00, A61B5/16","computer-aided systems and methods are provided for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles by statistically analyzing biological and/or psychological information collected from such user, such as age, gender, bodily sensati...","what we claim is:1. a computer-aided diagnosis system for determining the skin composition of a specific user according to traditional chinese medicinal (tcm) principles, said system comprising: (1) a data entry module configured for entering into said system a clinical data set obtained from t..."
37260,customized skin care products and personal care products based on the analysis of skin flora,"C12Q1/689, G16B30/00, G16B40/00","embodiments of the present invention relate to a combination of experimental and computational workflows that allow characterization of skin and subcutaneous tissue microbial flora and its associated metabolome, aiming to first evaluate an individual's skin and subcutaneous tissue to determine i...","1. a method for characterizing an individual who provides a microbiome, comprising: (a) generating a sequence profile comprising a nucleotide sequence associated with said microbiome; (b) generating a metabolome profile selected from the group consisting of a cellular growth rate associated with..."
48126,systems and methods for formulating personalized skincare products,"A45D44/00, A61B5/00, G06N5/04, G16H10/60, G16H20/10, G16H30/40, G16H50/20, G16H50/30","systems and methods for formulating a personalized skincare product for a user. data i

In [90]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

198

In [91]:
keyword = keywords_cosmetics[27]
keyword

'nail(s| |-)*(care|polish)'

In [92]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("G06F17")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
25707,image processing using a convolutional neural network to track a plurality of objects,"G06K9/62, G06N3/04, G06N3/08, G06T11/00, G06T11/60, G06T3/40, G06T7/11, G06T7/194, G06T7/246, G06T7/73","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","we claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn configu..."
45418,image processing using a convolutional neural network to track a plurality of objects,"G06N3/02, G06N3/08, G06T1/40, G06T7/10, G06T7/194","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","w e claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn config..."
52891,systems and methods for interactive virtual makeup experience,"G06F3/0484, G06F3/0488, G06T11/60, G06T7/136, G06T7/33, G06T7/60, G06T7/73, G06T7/90","a computing device for providing a virtual fingernail cosmetic experience generates an initial pattern located at a first position on a user interface displaying a digital image, the digital image further displaying a target object. the computing device obtains user input for relocating the init...","at least the following is claimed:1. a method implemented in a computing device having a processor, memory, and a display, the method for providing a virtual fingernail cosmetic experience, comprising: generating an initial pattern, the initial pattern being superimposed at a first position on a..."
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the invention proposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising: - acquiring an image of the body area without makeup of a subject, - determining first color parameters of the pixels of the image corresponding to the body area without makeup,...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
82678,methods of providing a patient with aesthetic improvement procedures,"G06Q10/00, G06Q50/00, G07G1/00",the invention relates to methods of providing a patient with aesthetic improvement procedures having two consultations. aesthetic improvement

In [93]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

207

In [94]:
keyword = keywords_cosmetics[28]
keyword

'baby(s| |-)*(care|polish)'

In [95]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("A47D15")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                  
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
106943,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/36, A61K8/368, A61Q15/00, A61Q19/10, C11B9/00, G06F17/10, G06F19/00, G06Q99/00",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."
134762,method of displaying products to consumers,G06Q30/00,a retail display for consumer products and method of display. a first packaged disposable absorbent product exhibiting on the package a first brand identifier and a second packaged disposable absorbent product exhibiting on the package a second brand identifier different from said first brand id...,"1. a method of displaying disposable absorbent products to a consumer in a retail environment, the method comprising the steps of: a. providing a first packaged disposable absorbent product exhibiting on the package a first brand identifier; b. providing a second packaged disposable absorbent ..."
164540,method of displaying products to consumers,"A47F7/00, A61F13/15, G09F3/00",a retail display for consumer products and method of display. a first packaged disposable absorbent product exhibiting on the package a first brand identifier and a second packaged disposable absorbent product exhibiting on the package a second brand identifier different from said first brand id...,"1. a method of displaying disposable absorbent products to a consumer in a retail environment, the method comprising the steps of: a. providing a first packaged disposable absorbent product exhibiting on the package a first brand identifier; b. providing a second packaged disposable absorbent pr..."
174990,method of making perfumed goods,"A61L9/01, A61Q19/00, C11B9/00, C11D3/22, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,what is claimed is:1. a method comprising:a.) using a malodour reduction value determined by mathematical modelling toselect one or more perfume raw materials;b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product. 2. the me...
175028,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/37, A61Q13/00, A61Q15/00, A61Q19/10, C11B9/00, C11D11/00, C11D3/50, G06F17/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."


In [96]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

213

In [97]:
keyword = keywords_cosmetics[29]
keyword

'anti(-)?ag(e|ing)'

In [98]:
df_candidate = df[(df[keyword] > 0)
                  
                   & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C12Q1")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06F17")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
66418,early detection of beauty treatment progress,"A45D44/00, A61B19/00, A61B5/00, A61B5/103, A61B5/107, G06F19/00, G06Q50/00, G06T11/00","a method of tracking progress of a beauty treatment regimen is disclosed. a typical individual undergoing the beauty treatment regimen may exhibit at least one indicia of progress. the indicia of progress may be substantially imperceptible to un-enhanced sensory faculties, such as vision and tou...","system for tracking progress of a beauty treatment regimen, the system comprising: means for prescribing the beauty treatment regimen to an individual, the prescribing means being suitable for altering a condition of at least one of skin, hair, teeth and nails, wherein a typical individual under..."
144514,early detection of beauty treatment progress,"A45D44/00, A61B19/00, A61B5/00, A61B5/103, A61B5/107, G06F19/00, G06Q50/00, G06T11/00","a method of tracking progress of a beauty treatment regimen is disclosed. a typical individual undergoing the beauty treatment regimen may exhibit at least one indicia of progress. the indicia of progress may be substantially imperceptible to un-enhanced sensory faculties, such as vision and tou...","1. a method of tracking progress of a beauty treatment regimen, the method comprising: prescribing the beauty treatment regimen to an individual, the regimen for altering a condition of at least one of skin, hair, teeth and nails, wherein a typical individual undergoing the beauty treatment regi..."
163843,identification and presentation of analogous beauty case histories,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","methods and systems are provided for assisting a subject in selecting a beauty product. in accordance with an embodiment, a method may include maintaining a plurality of sets of time-lapse body images for a plurality of individuals. in addition, an identity of at least one beauty product used by...","1. a method of assisting a subject in selecting at least one beauty product, the method comprising: maintaining a plurality of sets of time-lapse body images for a plurality of individuals; associating with each of the plurality of sets of time-lapse body images an identity of at least one beaut..."
213268,historical beauty record,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","systems, methods, and apparatus consistent with the present invention may be used to provide subjects with information reflecting cosmetic product usage. at least one cosmetic product may be distributed to a beauty facility. images of an external body condition of subject may be captured before ...","1. a beauty care method, comprising: obtaining first information representative of an external body condition of a subject; obtaining second information representative of the external body condition of the subject after the subject uses a beauty product; facilitating storage of information refle..."


In [99]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

217

In [100]:
keyword = keywords_cosmetics[30]
keyword

'colour|color'

In [101]:
df_candidate = df[(df[keyword] > 0)
                   & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B8")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61C7")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B1")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B29")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61M31")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61B34")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B5")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("F21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61M5/46")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("C09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A63F13/73")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61K31/444")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B66")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61M1/36")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61B3/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A43")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61B6/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G08")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61N5/10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61B3/12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61M21")==0) #Escludo "COMPUTING; CALCULATING; 
                   & (df.ipc_classes.str.count("B65")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47J31/44")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B26")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47J31/36")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                    & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61K31/403")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("D06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B64")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61K31/506")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47L15/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("E02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B32")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A22")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B67D1/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H03")==0) #Escludo "COMPUTING; CALCULATING; 
                  & (df.ipc_classes.str.count("F04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A41D1/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  
                 
                  
                  
                  
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
3232,processing content,,"a computer-implemented method of processing content items to extract information for analysis comprises implementing, at a data processing stage, the following steps: receiving the content items to be processed, wherein a plurality of probabilistic classifiers is executed at the data processing ...","1. a computer-implemented method of processing content items to extract information for analysis, the method comprising implementing, at a data processing stage, the following steps:receiving the content items to be processed, wherein a plurality of probabilistic classifiers is executed at the d..."
5337,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
13609,"method and apparatus for dynamic, real-time market segmentation",,"the invention involves generating and presenting, typically electronically, a number of design alternatives to persons who are participating in the design, selection, or market research exercise. the participants (referred to as 'selectors') transmit data indicative of their preferences among or...","claims what is claimed is:1. a method of dynamically identifying a subset of a set of items for which a plurality of selectors have a similar affinity, each of the items having a combination of attributes, the method comprising the steps of : (a) presenting for display to each of a group of sele..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
27424,method for traffic sign detection,,a method for detecting and recognizing at least one traffic sign is disclosed. a video sequence having a plurality of image frames is received. one or more filters are used to measure features in at least one image frame indicative of an object of interest. the measured features are combined and...,"we claim:1. a method for detecting and recognizing at least one traffic sign, the method comprising the steps of: a. receiving a video sequence comprised of a plurality of image frames; b. using one or more filters to measure features in at least one image frame indicative of an object of intere..."
27900,text to visualization,,"various implementations of the present disclosure relate to text to visualization. in a method, information items are extracted from a natural language sentence. visual elements associated with the information items are determined. a visual representation of the natural language sentence based o...","1. a device comprising:a processing unit;a memory coupled to the processing unit and comprising instructions stored thereon, the instructions, when executed by the processing unit, causing the device to perform acts comprising:extracting a plurality of information items from a natural language s..."
49306,robotic systems and methods for robustly grasping and targeting objects,,"embodiments are generally 

In [102]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

250

In [103]:
keyword = keywords_cosmetics[31]
keyword

'finger'

In [104]:
df_candidate = df[(df[keyword] > 0)
                  
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B8/14")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("H04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B60")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A62")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A47")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B32")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A22")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("F16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("D21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("D06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A23")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C04")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B64")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"  
                     & (df.ipc_classes.str.count("C12")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("F24")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("E21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G10")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61C7")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B1")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B29")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C40")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A61B3/16")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A61")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B65")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G09")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G08")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G11")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G03")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("E06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("F41")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("C07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("F21")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A01")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("A63")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("G02")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B25")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("A45C1/06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("H05")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B07")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   & (df.ipc_classes.str.count("B42F13/00")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("B66")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                   
                   
                 ].query("title not in @df_selected")
                 
    

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
11661,biological sequence fingerprints,,"in accordance with one embodiment of the invention, features of biological sequences are represented in a fingerprint that includes a bitset, and may also include counts, strings or continuous values, for the features. the fingerprint can be used with machine learning and statistical methods. th...","1. a computer-implemented method for forming a fingerprint data structure representing a biological sequence, the computer-implemented method comprising: for each component feature of a plurality of component features to be used in the fingerprint data structure, querying a biological sequence d..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
27426,personal care device,B26B19/38,"the present invention relates to a personal care device, in particular skin treatment device such as electric shaver, comprising an elongated handle for manually moving the personal care device along a body surface, a working head attached to said handle for effecting a personal care treatment t...","1. personal care device, in particular hair removal device such as electric shaver, comprising an elongated handle (2) for manually moving the personal care device along a body surface, a working head (3) attached to said handle (2) for effecting a personal care treatment to said body surface, a..."
29669,personal care device,B26B19/38,"the present invention relates to a personal care device, in particular skin treatment device such as electric shaver, comprising an elongated handle for manually moving the personal care device along a body surface, a working head attached to said handle for effecting a personal care treatment t...","1. personal care device, in particular hair removal device such as electric shaver, comprising an elongated handle (2) for manually moving the personal care device along a body surface, a working head (3) attached to said handle (2) for effecting a personal care treatment to said body surface, a..."
41070,personal care device,"B26B19/04, B26B19/10, B26B19/28, B26B19/38, B26B19/40, B26B19/46","an electric shaver, comprising an elongated handle for manually moving the personal care device along a body surface, a working head attached to said handle for effecting a personal care treatment to said body surface, at least one detector for detecting at least one behavioral parameter indicat...","1. a electric shaver as a personal care device, comprising an elongated handle for manually moving the personal care device along a body surface, a working head attached to said handle for effecting a personal care treatment to said body surface, at least one detector for detecting at least one ..."
49306,robotic systems and methods for robustly grasping and targeting objects,,"embodiments are generally directed to generating a training dataset of labelled examples of sensor images and grasp configurations using a set of three-dimensional (3d) models of objects, one or more analytic mechanical representations of either or both of grasp forces and grasp torques, and sta...","1. a computerimplemented method, comprising:generating a training dataset of labelled examples of sensor images and grasp configurations using:a set of three-dimensional (3d) models of objects;one or more analytic mechanical representations of

In [105]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

263

In [106]:
#method for simulating the rendering of a make-up product on a body area	
#method for simulating the rendering of a make-up product on a body area	
#methods and compositions for treating pain	A61K31/167, A61K31/192, A61K31/198, A61K31/428
#191501	method and system for improving infection control in a facility	G08B21/24, 
#88971	methods and compositions for identification and modulation of microbiome biochemical pathways to alter phenotype	A61K31/015, A61K31/03, A61K31/11, A61K31/20, A61K9/00


In [108]:
len(df_selected)

263

In [110]:
len(df_selected.drop_duplicates())

207

In [111]:
df_selected.drop_duplicates()[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","a system for dispensing at least one makeup product, including a cup and a dispenser for filling the cup from the bottom with at least one product, the cup being secured to the dispenser at least while it is being filled with said product. the method for dispensing and evaluating makeup, includi...","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
31462,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04",a beauty product device is disclosed which includes a variety of sensors for sensing the way in which a user is operating the device. the sensor data can be processed and compared with pre-stored data representative of a desired style and feedback messages generated and provided back to the user...,"1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
38540,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","the invention is a cosmetic cleaning product, comprising a massage applicator comprising a container body and a massage device having at least one massage pin device, the pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a ...","1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a force ca..."
53114,portable apparatus for supporting a hand held hair dryer,"A45D20/12, A45D20/14, F16M11/00, F16M13/02",a portable hair dryer support apparatus comprising a hair dryer holding structure for holding a hair dryer and at least one vertical support element connected at a first end to the hair dryer holding structure and having at a second opposed end at least one securing portion for securing the port...,"1. a portable hair dryer support apparatus, comprising: a hair dryer holding structure for holding a hair dryer; at least one vertical support element pivotally and directly connected at a bottom end to the hair dryer holding structure via a hinge having an axis of rotation perpendicular to the ..."
53136,improvements in and relating to hair irons,"A45D1/06, A45D1/28",there are provided hair irons. in a preferred embodiment an electric hair iron (1) comprises at least one heating element and control means comprising a temperature sensor and a control circuit. the control means is arranged such that power is only supplied 